In [1]:
import pandas as pd
import numpy as np

# Make dummy data

In [2]:
age = ["10代", "20代", "30代", "40代", "50代", "60代", "70代"]
gender = ["女性", "男性"]

In [3]:
def generate_users(n_users, **features):
    """make user list where columns are 'user_id' and feature columns ('age', 'gender',... etc)
       values of the feature columns are randomly chosen from the given list
    """
    user_id = [i + 1 for i in range(n_users)]
    df = pd.DataFrame({"user_id":user_id})
    
    np.random.seed(100)
    for col in features:
        feature = features[col]
        values = np.random.choice(feature, n_users)
        df[col] = values
    
    return df


def add_flag(df, **targets):
    """add flag column named as 'x_flag'
       bias is added to the users specified in the variables '**targets'
    """
    n_users = df.shape[0]
    
    # specify users to whom bias is added
    is_target = [True for _ in range(n_users)]
    for col in targets:
        target_values = targets[col]
        is_value = [False for _ in range(n_users)]    
        for value in target_values:
            tmp_bool = (df[col] == value)
            is_value = np.logical_or(is_value, tmp_bool)
        is_target = np.logical_and(is_target, is_value)
            
    #assign flag
    flag = [0, 1]
    #for ordinary users, 10% of them are assigned to 'flag=1'
    weight = [0.9, 0.1]
    np.random.seed(200)
    df["x_flag"] = np.random.choice(flag, n_users, p=weight)
    #for users specified above, 90% of them are assigned to 'flag=1'  
    weight = [0.1, 0.9]
    np.random.seed(300)
    df.ix[is_target, "x_flag"] = np.random.choice(flag, n_users, p=weight)
    
    return df
    
    
def add_rate(df, **targets):
    """add real valued column named as 'x_rate', where the values are normalized between 0 and 1
       for the users with 'flag=1', values in 'x_rate' are increased
       for the users specified by '**targets', values in 'x_rate' are much higher, regardless of 'flag=1 or 0'
    """
    n_users = df.shape[0]
    
    #specify users to whom high 'x_rate' values are set
    is_target = [True for _ in range(n_users)]
    for col in targets:
        target_values = targets[col]
        is_value = [False for _ in range(n_users)]    
        for value in target_values:
            tmp_bool = (df[col] == value)
            is_value = np.logical_or(is_value, tmp_bool)
        is_target = np.logical_and(is_target, is_value)
            
    #for users with 'flag=0', 'x_rate' values are sampled from Normal(0.2, 0.1)
    np.random.seed(200)
    df["x_rate"] = np.random.normal(0.2, 0.1, n_users)
    
    #for users with 'flag=1', 'x_rate' values are increased by 0.2 point
    is_flag = (df["x_flag"] == 1)
    df.ix[is_flag, "x_rate"] += 0.2
    
    #for users specified by '**targets', 'x_rate' values are sampled from Normal(0.8, 0.2)
    np.random.seed(300)
    df.ix[is_target, "x_rate"] = np.random.normal(0.8, 0.2, n_users)
    
    #for convenience normalize to [0 ,1]
    df.ix[df["x_rate"] > 1, "x_rate"] = 1    
    df.ix[df["x_rate"] < 0, "x_rate"] = 0

    return df    

In [4]:
n_users = 100000
df = generate_users(n_users, age=age, gender=gender)
df = add_flag(df, age=["20代", "30代"], gender=["男性"])
df = add_rate(df, age=["20代", "30代"], gender=["男性"])

# Check the data

In [5]:
df.head(10)

,user_id,age,gender,x_flag,x_rate
0,1,10代,男性,1,0.254905
1,2,10代,女性,0,0.391095
2,3,40代,女性,0,0.271188
3,4,10代,男性,0,0.175226
4,5,30代,男性,1,0.502966
5,6,70代,男性,0,0.196705
6,7,50代,男性,0,0.177865
7,8,30代,女性,1,0.447726
8,9,60代,女性,0,0.130806
9,10,30代,女性,1,0.479201


## the number of users

In [6]:
df_count = df.groupby(["x_flag", "gender", "age"])["user_id"].count().unstack(level=[1,2]).sort_index(ascending=False)
df_count.ix["ratio of 1 (%)"] = 100 * df_count.ix[1, :] / (df_count.ix[1, :] + df_count.ix[0, :])
df_count.astype(int)

gender            女性                                        男性              \
age              10代   20代   30代   40代   50代   60代   70代   10代   20代   30代   
x_flag                                                                       
1                734   708   742   712   672   748   696   707  6380  6406   
0               6589  6352  6395  6539  6523  6415  6379  6431   720   676   
ratio of 1 (%)    10    10    10     9     9    10     9     9    89    90   

gender                                  
age              40代   50代   60代   70代  
x_flag                                  
1                716   741   688   693  
0               6480  6477  6293  6388  
ratio of 1 (%)     9    10     9     9

## mean of the rate

In [7]:
df_rate = df.groupby(["x_flag", "gender", "age"])["x_rate"].mean().unstack(level=[1,2]).sort_index(ascending=False)
df_rate.ix['diff (1-0)', :] = df_rate.ix[1, :] - df_rate.ix[0, :]
df_rate.round(3)

gender         女性                                               男性         \
age           10代    20代    30代    40代    50代    60代    70代    10代    20代   
x_flag                                                                      
1           0.402  0.399  0.402  0.397  0.397  0.399  0.402  0.398  0.786   
0           0.201  0.200  0.203  0.204  0.201  0.199  0.199  0.200  0.782   
diff (1-0)  0.201  0.200  0.199  0.193  0.196  0.200  0.203  0.197  0.004   

gender                                         
age           30代    40代    50代    60代    70代  
x_flag                                         
1           0.778  0.400  0.403  0.403  0.405  
0           0.779  0.199  0.202  0.199  0.199  
diff (1-0) -0.001  0.201  0.200  0.204  0.205

## Estimate the Naive Expected Effect

In [8]:
E1 = np.mean(df.ix[df["x_flag"] == 1, "x_rate"])
E0 = np.mean(df.ix[df["x_flag"] == 0, "x_rate"])
E = E1 - E0
print("Expected Effect = {0}   (x_flag=1 : {1}, x_flag=0 : {2}) ".format(round(E, 3), round(E1, 3), round(E0, 4)))

Expected Effect = 0.418   (x_flag=1 : 0.629, x_flag=0 : 0.2109) 


# Analysis with Propensity Score

In [9]:
from sklearn.linear_model import LogisticRegression 

In [20]:
class PropensityScore:
    def __init__(selef, df):
        selef.df_data = df.copy()
        
    def fit(self, X_vars):
        #calculate propensity score 'e' by Logistic Regression
        self.model = LogisticRegression() 
        self.X = pd.get_dummies(self.df_data[X_vars]).values
        self.y = self.df_data["x_flag"].values
        self.model.fit(self.X, self.y)
        self.df_data["e"] = self.model.predict_proba(self.X)[:, np.where(self.model.classes_ == 1)].flatten()
        
    def estimate(self):
        #calculate 'IPWE'(Inverse Probability Weighting Estimator)
        w1 = self.df_data["x_flag"] / self.df_data["e"]
        w0 = (1 - self.df_data["x_flag"]) / (1 - self.df_data["e"]) 
        self.E1 = (np.sum(self.df_data["x_rate"] * w1)) / np.sum(w1) # E(y1)
        self.E0 = (np.sum(self.df_data["x_rate"] * w0)) / np.sum(w0) # E(y0)
        
        #calculate other expectation values
        w1 = self.df_data["x_flag"]
        w0 = (1 - self.df_data["x_flag"])
        self.E11 = (np.sum(self.df_data["x_rate"] * w1)) / np.sum(w1) # E(y1|z=1)
        self.E00 = (np.sum(self.df_data["x_rate"] * w0)) / np.sum(w0) # E(y0|z=0)
        self.p1 = np.sum(w1) / np.sum(w0 + w1) # p(z=1)
        self.p0 = np.sum(w0) / np.sum(w0 + w1) # p(z=0)
        
        self.E10 = (self.E1 - self.E11 * self.p1) / self.p0 if self.p0 != 0 else None # E(y1|z=0)
        self.E01 = (self.E0 - self.E00 * self.p0) / self.p1 if self.p1 != 0 else None # E(y0|z=1)
        
        self.ATE = self.E1 - self.E0   # E(y1 - y0)
        self.ATT = self.E11 - self.E01 # E(y1 - y0|z=1)
        self.ATU = self.E10 - self.E00 # E(y1 - y0|z=0)

In [21]:
ps = PropensityScore(df)

In [22]:
ps.fit(["age", "gender"])
ps.estimate()

In [26]:
print("ATE = E(y1 - y0) = {0}".format(round(ps.ATE, 3)))

ATE = E(y1 - y0) = 0.209


In [25]:
print("ATT = E(y1 - y0|z=1) = {0}".format(round(ps.ATT, 3)))
print("ATU = E(y1 - y0|z=0) = {0}".format(round(ps.ATU, 3)))

ATT = E(y1 - y0|z=1) = 0.321
ATU = E(y1 - y0|z=0) = 0.178


In [27]:
ps.p1

0.21343000000000001

In [28]:
ps.p0

0.78656999999999999

In [29]:
ps.ATT * ps.p1 + ps.ATU * ps.p0

0.20888280297641398

In [35]:
ps.E11

0.6289945371017629

In [31]:
ps.E01

0.30784832494047504

In [32]:
ps.E00

0.21093636450796127

In [34]:
ps.E10

0.38935731485545777